# Finding best model and hyper parameter tunning using GridSearchCV


### For iris flower dataset in sklearn library, we are going to find out best model and best hyper parameters using GridSearchCV

In [17]:
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()

In [4]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x:iris.target_names[x])
df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


# Approach 1: Use train_test_split and manually tune parameters by trial and error


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [9]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train,y_train)
model.score(X_test,y_test)

1.0

# Approach 2: Use K Fold Cross validation


### Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation



In [10]:
from sklearn.model_selection import cross_val_score

In [12]:
cross_val_score(SVC(kernel='linear',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [13]:
cross_val_score(SVC(kernel='rbf',C=10,gamma='auto'),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [15]:
cross_val_score(SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

#### Above approach is tiresome and very manual. We can use for loop as an alternative



In [19]:
kernels = ['rbf','linear']
C = [1,10,20]
avg_scores = {}
for k_value in kernels:
    for c_value in C:
        cross_score = cross_val_score(SVC(kernel=k_value,C=c_value,gamma='auto'),iris.data,iris.target,cv=5)
        avg_scores[k_value + '_' + str(c_value)] = np.average(cross_score)
avg_scores


{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

#### From above results we can say that rbf with C=1 or 10 or linear with C=1 will give best performance

# Approach 3: Use GridSearchCV

### GridSearchCV does exactly same thing as for loop above but in a single line of code

In [23]:
from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(SVC(gamma='auto'),{'kernel':['rbf','linear'],'C':[1,10,20]},cv=5,return_train_score=False)
gs.fit(iris.data,iris.target)
gs.cv_results_

{'mean_fit_time': array([0.00060334, 0.        , 0.00030088, 0.00080285, 0.0007966 ,
        0.00079799]),
 'std_fit_time': array([0.00080073, 0.        , 0.00060177, 0.00040164, 0.00039831,
        0.00039899]),
 'mean_score_time': array([0.0035284 , 0.00181446, 0.        , 0.00059381, 0.00079207,
        0.00059824]),
 'std_score_time': array([0.00562148, 0.00362892, 0.        , 0.000485  , 0.00039626,
        0.00048846]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [24]:
df = pd.DataFrame(gs.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000603,0.000801,0.003528,0.005621,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000000,0.000000,0.001814,0.003629,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.000301,0.000602,0.000000,0.000000,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.000803,0.000402,0.000594,0.000485,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.000797,0.000398,0.000792,0.000396,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000798,0.000399,0.000598,0.000488,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [25]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [28]:
gs.best_params_

{'C': 1, 'kernel': 'rbf'}

In [29]:
gs.best_score_

0.9800000000000001

In [30]:
dir(gs)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_feature_names',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_metadata_request',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 '_validate_params',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 '

#### Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation



In [34]:
from sklearn.model_selection import RandomizedSearchCV
rs =RandomizedSearchCV(SVC(gamma='auto'),{'kernel':['rbf','linear'],'C':[1,10,20]},cv=5,return_train_score=False,n_iter=2)
rs.fit(iris.data,iris.target)
rs.cv_results_

{'mean_fit_time': array([0.00182467, 0.        ]),
 'std_fit_time': array([0.00041984, 0.        ]),
 'mean_score_time': array([0.00099759, 0.        ]),
 'std_score_time': array([0.00063075, 0.        ]),
 'param_kernel': masked_array(data=['linear', 'linear'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_C': masked_array(data=[1, 20],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel': 'linear', 'C': 1}, {'kernel': 'linear', 'C': 20}],
 'split0_test_score': array([0.96666667, 1.        ]),
 'split1_test_score': array([1., 1.]),
 'split2_test_score': array([0.96666667, 0.9       ]),
 'split3_test_score': array([0.96666667, 0.93333333]),
 'split4_test_score': array([1., 1.]),
 'mean_test_score': array([0.98      , 0.96666667]),
 'std_test_score': array([0.01632993, 0.0421637 ]),
 'rank_test_score': array([1, 2])}

In [36]:
df1 = pd.DataFrame(rs.cv_results_)
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001825,0.00042,0.000998,0.000631,linear,1,"{'kernel': 'linear', 'C': 1}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000000,0.00000,0.000000,0.000000,linear,20,"{'kernel': 'linear', 'C': 20}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,2


### How about different models with different hyperparameters?

In [37]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}

In [38]:
scores = []

for model_name, mp in model_params.items():
    gs =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    gs.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': gs.best_score_,
        'best_params': gs.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.953333,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}


### Based on above, I can conclude that SVM with C=1 and kernel='rbf' is the best model for solving my problem of iris flower classification